# Plane Wave Visualization

In this notebook, we attempt to visualize plane wave travel and reflection. You might the format to be reminiscent of the transmission line reflections notebook.

## Traveling Plane Wave Without Reflections

First, we start with visualizing a forward traveling EM wave (visualizing the $\vec{E}$ component without reflections. The top pane shows the E field intensity of an EM wave traveling in the $+z$ direction; the bottom pane plots the cross-sectional field amplitude.

In [14]:
# reset the Python environment
%reset -f

# import needed modules again
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import HTML

# set plot params
# NOTE: the code treats the horizontal axis as "x" although it is labelled "z".
# This is due to reusing code from prior Notebooks without wanting to rename all 
# of the variables.
x_min, x_max = 0, 1
y_min, y_max = -2, 2
z_min, z_max = 0, 1
x_res, y_res, z_res = 250,250,250   # discrete points per unit dim

# animation settings
anim_interval = 25  # delay between animation frames (ms)
anim_period = 6     # animation period (s)
anim_frames = int(1e3 * anim_period / anim_interval)

In [8]:
def map_func(x, z, t):
    '''
    Return a tuple with the 1st element containing a 2-d list representing the wave amplitudes
    at the given time t, and the 2nd element containing a 1-d list representing the wave amplitudes
    at a cross section for a constant z value.
    '''
    eta = lambda x, x0, lmda, v: 2*np.pi/lmda*(x-x0-v*t)
    sigmoid = lambda p,x0: 1 / (1 + np.exp(p*(x-x0)))
    wf = lambda x0,lmda,v: sigmoid(30, x0+v*t)*np.sin(eta(x,x0,lmda,v)) + \
            refl_coef*sigmoid(-30, 2*x_max-(x0+v*t))*np.sin(eta(x,2*x_max,lmda,-v))
    global v_min
    global v_max
    x0, lmda, v = 0, 0.2, 0.2
    im = [sigmoid(30, x0+v*t)*np.sin(eta(x,x0,lmda,v)) for i in range(len(z))]
    return im, im[0]

In [9]:
# reflection coefficient
refl_coef = -1

def init():
    '''Re-initialize the line plot, expected to be called by FuncAnimation'''
    line.set_data([],[])
    im.set_data(np.zeros([xdim,zdim]))
    return (im, line)

def animate_line(t):
    '''
    Set the line's x and y plotting points at the specified time step.
    
    Parameters:
        t: discrete time step
    
    Returns:
        Matplotlib line in tuple form.
    '''
    imd, ld = map_func(x,z,t)
    line.set_data(x, ld)
    im.set_data(imd)
    return (im, line)

# needed arrays
t_vals = np.linspace(0,anim_period,anim_frames)
x = np.linspace(x_min, x_max, x_res*x_max)
z = np.linspace(z_min, z_max, z_res*z_max)
xdim, zdim = len(x), len(z)

# construct maps for animation
# imaps = []
# cxs = []
# for t in np.linspace(0,anim_period,anim_frames):
#     # get the map (m) and cross section (cx) data
#     m,cx = map_func(x, z, t)
#     imaps.append(m)
#     cxs.append(cx)

# imshow properties
fig, [ax0,ax1] = plt.subplots(2, 1, figsize=(10,10))
divider0 = make_axes_locatable(ax0)
ax0_cb = divider0.append_axes('right', size='5%', pad=0.05)
ax0.set_title('Forward Travelling 2-d Plane Wave')
ax0.set_xlabel(r'$z$ (m)')
ax0.set_ylabel(r'$x$ (m)')

# color bar for 2-d map
cmap = mpl.cm.bwr
norm = mpl.colors.Normalize(vmin=y_min, vmax=y_max)
cb0 = mpl.colorbar.ColorbarBase(ax0_cb, cmap=cmap, norm=norm, orientation='vertical')

# line plot properties
divider1 = make_axes_locatable(ax1)
ax1_cb = divider1.append_axes('right', size='5%', pad=0.05)
ax1_cb.axis('off')
ax1.set_title('Cross-section E Field Amplitude')
ax1.set_xlabel(r'$z$ (m)')
ax1.set_ylabel(r'$\mathcal{E}$ (V/m)')
ax1.set_xlim((x_min,x_max))
ax1.set_ylim((y_min,y_max))

# plots
im = ax0.imshow(np.zeros([xdim,zdim]), cmap=cmap, norm=norm,
                extent=(x_min, x_max, z_min, z_max),
                animated=True, aspect='auto')
line, = ax1.plot([], [], c='tab:blue')

# generate 2-d animated map and cross section view
# artists = []
# for imap, cx in zip(imaps, cxs):
#     im = ax0.imshow(imap, cmap=cmap, norm=norm,
#                    extent=(x_min, x_max, z_min, z_max),
#                    animated=True, aspect='auto')
#     cp, = ax1.plot(x, cx, c='tab:blue')
#     artists.append([im,cp])

# animate
# anim = animation.ArtistAnimation(fig, artists, interval=anim_interval, blit=True)
anim = animation.FuncAnimation(fig, animate_line, init_func=init, frames=t_vals,
                               interval=anim_interval, blit=True)
plt.close()
HTML(anim.to_html5_video())
#HTML(anim.to_jshtml())

## Adding Reflection to the Mix

Now let's add a reflection boundary at the right of the plot ($z = 1.0$). Tune the variable `refl_coef` to see how the wave behaves!

Does this align with your expectations having learned about transmission line reflections? Take some time to mentally correlate the plane wave 2D visualization with the amplitude plot.

In [21]:
# reset the Python environment
%reset -f

# import needed modules again
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import HTML

# set plot params
# NOTE: the code treats the horizontal axis as "x" although it is labelled "z".
# This is due to reusing code from prior Notebooks without wanting to rename all 
# of the variables.
x_min, x_max = 0, 1
y_min, y_max = -2, 2
z_min, z_max = 0, 1
x_res, y_res, z_res = 250,250,2     # discrete points per unit dim

In [22]:
v_min, v_max = float('inf'), -float('inf')

def map_func(x, z, t):
    '''
    Return a tuple with the 1st element containing a 2-d list representing the wave amplitudes
    at the given time t, and the 2nd element containing a 1-d list representing the wave amplitudes
    at a cross section for a constant z value.
    '''
    eta = lambda x, x0, lmda, v: 2*np.pi/lmda*(x-x0-v*t)
    sigmoid = lambda p,x0: 1 / (1 + np.exp(p*(x-x0)))
    wf = lambda x0,lmda,v: sigmoid(30, x0+v*t)*np.sin(eta(x,x0,lmda,v)) - \
            refl_coef*sigmoid(-30, 2*x_max-(x0+v*t))*np.sin(eta(x,2*x_max,lmda,-v))
    global v_min
    global v_max
    x0, lmda, v = 0, 0.2, 0.2
    im = [wf(x0,lmda,v) for i in range(len(z))]
    v_min = min(v_min, np.amin(im))
    v_max = max(v_max, np.amax(im))
    return im, im[0]

In [23]:
# reflection coefficient
refl_coef = 0.3

# animation settings
anim_interval = 33  # delay between animation frames (ms)
anim_period = 12    # animation period (s)
anim_frames = int(1e3 * anim_period / anim_interval)

# needed arrays
x = np.linspace(x_min, x_max, x_res*x_max)
z = np.linspace(z_min, z_max, z_res*z_max)

# construct maps for animation
imaps = []
cxs = []
for t in np.linspace(0,anim_period,anim_frames):
    # get the map (m) and cross section (cx) data
    m,cx = map_func(x, z, t)
    imaps.append(m)
    cxs.append(cx)

# imshow properties
fig, [ax0,ax1] = plt.subplots(2, 1, figsize=(10,10))
divider0 = make_axes_locatable(ax0)
ax0_cb = divider0.append_axes('right', size='5%', pad=0.05)
ax0.set_title('Forward Travelling 2-d Plane Wave')
ax0.set_xlabel(r'$z$ (m)')
ax0.set_ylabel(r'$x$ (m)')
# color bar for 2-d map
cmap = mpl.cm.bwr
norm = mpl.colors.Normalize(vmin=v_min, vmax=v_max)
cb0 = mpl.colorbar.ColorbarBase(ax0_cb, cmap=cmap, norm=norm, orientation='vertical')

# line plot properties
divider1 = make_axes_locatable(ax1)
ax1_cb = divider1.append_axes('right', size='5%', pad=0.05)
ax1_cb.axis('off')
ax1.set_title('Cross-section E Field Amplitude')
ax1.set_xlabel(r'$z$ (m)')
ax1.set_ylabel(r'$\mathcal{E}$ (V/m)')
ax1.set_xlim((x_min,x_max))
ax1.set_ylim((y_min,y_max))


# generate 2-d animated map and cross section view
artists = []
for imap, cx in zip(imaps, cxs):
    im = ax0.imshow(imap, cmap=cmap, norm=norm,
                   extent=(x_min, x_max, z_min, z_max),
                   animated=True, aspect='auto')
    cp, = ax1.plot(x, cx, c='tab:blue')
    artists.append([im,cp])

# animate
anim = animation.ArtistAnimation(fig, artists, interval=anim_interval, blit=True)
plt.close()
HTML(anim.to_html5_video())

## Steady State Reflection and Transmission

Lastly, we look at a steady state simulation. Once again, tune the `refl_coef` parameter to visualize various reflection conditions. Does this again align with transmission line understanding?

In [36]:
# reset the Python environment
%reset -f

# import needed modules again
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import HTML

# set plot params
# NOTE: the code treats the horizontal axis as "x" although it is labelled "z".
# This is due to reusing code from prior Notebooks without wanting to rename all 
# of the variables.
x_min, x_max = 0, 2
y_min, y_max = -2, 2
z_min, z_max = 0, 1
x_res, y_res, z_res = 250,250,250   # discrete points per unit dim

In [37]:
v_min, v_max = float('inf'), -float('inf')

def map_func(x, z, t):
    '''
    Return a tuple with the 1st element containing a 2-d list representing the wave amplitudes
    at the given time t, and the 2nd element containing a 1-d list representing the wave amplitudes
    at a cross section for a constant z value.
    '''
    global v_min
    global v_max
    eta = lambda x, x0, lmda, v: 2*np.pi/lmda*(x-x0-v*t)
    
    lmda, v = 0.2, 0.1
    
    w = {}
    w['i'] = np.sin(eta(x,0,lmda,v)) * np.heaviside(-x+x_max/2,0)
    w['r'] = -refl_coef*np.sin(eta(x,x_max/2,lmda,-v)) * np.heaviside(-x+x_max/2,0)
    w['t'] = (1+refl_coef)*np.sin(eta(x,x_max/2,lmda,v)) * np.heaviside(x-x_max/2,1)
    w['a'] = w['i'] + w['r'] + w['t']
    
    im = [w['a'] for i in range(len(z))]
    v_min = min(v_min, np.amin(im))
    v_max = max(v_max, np.amax(im))
    return im, im[0]

In [38]:
# reflection coefficient
refl_coef = -0.3

# animation settings
anim_interval = 33  # delay between animation frames (ms)
anim_period = 2    # animation period (s)
anim_frames = int(1e3 * anim_period / anim_interval)

# needed arrays
x = np.linspace(x_min, x_max, x_res*x_max)
z = np.linspace(z_min, z_max, z_res*z_max)

# construct maps for animation
imaps = []
cxs = []
for t in np.linspace(0,anim_period,anim_frames):
    # get the map (m) and cross section (cx) data
    m,cx = map_func(x, z, t)
    imaps.append(m)
    cxs.append(cx)

# imshow properties
fig, [ax0,ax1] = plt.subplots(2, 1, figsize=(10,10))
divider0 = make_axes_locatable(ax0)
ax0_cb = divider0.append_axes('right', size='5%', pad=0.05)
ax0.set_title(rf'Steady State Reflection and Transmission of 2-d Plane EM Wave with $\Gamma = {refl_coef}$')
ax0.set_xlabel(r'$z$ (m)')
ax0.set_ylabel(r'$x$ (m)')
# color bar for 2-d map
cmap = mpl.cm.bwr
norm = mpl.colors.Normalize(vmin=v_min, vmax=v_max)
cb0 = mpl.colorbar.ColorbarBase(ax0_cb, cmap=cmap, norm=norm, orientation='vertical')

# line plot properties
divider1 = make_axes_locatable(ax1)
ax1_cb = divider1.append_axes('right', size='5%', pad=0.05)
ax1_cb.axis('off')
ax1.set_title('Cross-section E Field Amplitude')
ax1.set_xlabel(r'$z$ (m)')
ax1.set_ylabel(r'$\mathcal{E}$ (V/m)')
ax1.set_xlim((x_min,x_max))
ax1.set_ylim((y_min,y_max))

# generate 2-d animated map and cross section view
artists = []
for imap, cx in zip(imaps, cxs):
    im = ax0.imshow(imap, cmap=cmap, norm=norm,
                   extent=(x_min, x_max, z_min, z_max),
                   animated=True, aspect='auto')
    cp, = ax1.plot(x, cx, c='tab:blue')
    artists.append([im,cp])
ax0.axvline(x=1, c='black')
ax1.axvline(x=1, c='black')

# animate
anim = animation.ArtistAnimation(fig, artists, interval=anim_interval, blit=True)
plt.close()
HTML(anim.to_html5_video())